In [ ]:
import sys, os, datetime
import numpy as np
from dwave_qbsolv import QBSolv
import neal
import itertools
import math
import time
from pyqubo import Binary, Array
from python_fjda import fjda, da_verify
N = 1024
# calculating absolute distance between two points
def abs_distance(x, y):
    return abs(x - y)


def euclidean_distance(x1, x2, y1, y2):
    return pow(pow(x1 - x2, 2) + pow(y1 - y2, 2), 0.5)


# Checking and setting wake influence between pairs of cells
def check_wake(p_i, p_j):
    alpha = 0.09437
    rotor_radius = 27.881
    
    if p_j[1] > p_i[1]:
        #xdistance = abs_distance(p_i[0], p_j[0])
        ydistance = abs_distance(p_i[1], p_j[1])

        radius = rotor_radius + (alpha * ydistance)

        xmin = p_j[0] - radius
        xmax = p_j[0] + radius
    else:
        return False

    if (xmin < p_i[0] and xmax > p_i[0]):
        return True
    else:
        return False


# Calculating velocity factor between two cells due to wakes
def calc_velocity(loc_y_i, loc_y_j, u_inf):
    alpha = 0.09437
    a = 0.326795
    rotor_radius = 27.881

    # todo: check proper calculation of wind
    ydistance = abs_distance(loc_y_i,
                             loc_y_j)  # euclidean_distance(0,0,loc_y_i, loc_y_j) #abs_distance(loc_y_i, loc_y_j)
    denom = pow((alpha * ydistance / rotor_radius) + 1, 2)

    return u_inf * (1 - (2 * a / denom))


def is_in_grid(x, y, max_x, max_y):
    if (x < 0 or x > max_x) or (y < 0 or y > max_y):
        return False
    else:
        return True


# setting cell locations:
def calc_location(axis_, step):
    location_x = []
    location_y = []
    for x in range(0, axis_):
        for y in range(0, axis_):
            location_x.append(x * step)
            location_y.append(y * step)
    return location_x, location_y

def set_wind_regime(n_wind):
    wind_speeds = []
    prob_l = []
    if n_wind == 36:
        wind_speeds = np.array([8] * n_wind)
        wind_speeds = np.append(wind_speeds, [12] * n_wind)
        wind_speeds = np.append(wind_speeds, [17] * n_wind)

        prob_l = np.array([float(0.0047)] * n_wind)
        prob_l = np.append(prob_l, [0.008] * (n_wind - 9))  # 9 special regimes
        prob_l = np.append(prob_l, [0.01, 0.012, 0.0145, 0.014, 0.019, 0.014, 0.0145, 0.012, 0.01])
        prob_l = np.append(prob_l, [0.011] * (n_wind - 9))
        prob_l = np.append(prob_l, [0.013, 0.017, 0.0185, 0.03, 0.035, 0.03, 0.0185, 0.017, 0.013])
    elif n_wind ==1:


        wind_speeds = np.array([12] * n_wind)

        prob_l = np.array([1])

    return wind_speeds, prob_l

def calc_coefficients(location_x, location_y, wind_speeds, prob_l, n):
    # Epsilon contains pairs of cells where Turbines cannot be placed due to proximity.
    # Smaller than dist x step (meters) is not allowed. Absolute distance is used.
    Epsilon = []
    #dist = 1  # 100 meters between turbines (in some papers it is 200 meters so dist should be 2)
    for i in range(n):
        for j in range(n):
            if ((abs_distance(location_x[i], location_x[j]) < 200) and (
                    abs_distance(location_y[i], location_y[j]) < 200)) and (i != j) and (j, i) not in Epsilon:
                Epsilon.append((i, j))
    len(Epsilon)

    max_x = max(location_x)
    max_y = max(location_y)
    velocity = np.zeros((n, n, len(wind_speeds)))
    energy_coef = np.zeros((n, n, len(wind_speeds)))
    ss_coef = np.zeros((n, n, len(wind_speeds)))
    U = {}
    # per each cell, we want to consider the locations that if turbine is placed there it will result
    # in a wake
    for l in range(0, len(wind_speeds)):
        for i in range(n):
            U[(i, l)] = []
            for j in range(n):
                # print(str((i,j)))
                # default influence is zero (no wake)
                energy_coef[i, j, l] = 0
                if i == j:
                    continue
                # assuming wind is going south to north, wakes will happen only on the y axis bottom up
                # if location_y[i]>=location_y[j]:
                # if true, then i is northern to j so it is influenced by it but does not influence it (j is upstream to i)
                #    continue
                # else:
                # potential wake
                temp_x_i = location_x[i] * math.cos(((l % n_wind) * angle) * math.pi / 180) - location_y[i] * math.sin(
                    ((l % n_wind) * angle) * math.pi / 180)

                temp_x_j = location_x[j] * math.cos(((l % n_wind) * angle) * math.pi / 180) - location_y[j] * math.sin(
                    ((l % n_wind) * angle) * math.pi / 180)

                temp_y_i = location_x[i] * math.sin(((l % n_wind) * angle) * math.pi / 180) + location_y[i] * math.cos(
                    ((l % n_wind) * angle) * math.pi / 180)
                temp_y_j = location_x[j] * math.sin(((l % n_wind) * angle) * math.pi / 180) + location_y[j] * math.cos(
                    ((l % n_wind) * angle) * math.pi / 180)

                # print(temp_x_i, temp_x_j, temp_y_i, temp_y_j)
                # diff = euclidean_distance(location_x[i], location_x[j], location_y[i], location_y[j]) -\
                #             euclidean_distance(temp_x_i, temp_x_j, temp_y_i, temp_y_j)

                if True:  # is_in_grid(temp_x_i, temp_y_i, max_x, max_y) and is_in_grid(temp_x_j, temp_y_j, max_x, max_y):
                    # print((l,i,j))

                    prob = prob_l[l]
                    u_inf = wind_speeds[l]
                    if check_wake((temp_x_i, temp_y_i), (temp_x_j, temp_y_j)):
                        # if check_wake((0,0), (temp_x_j, temp_y_j)):
                        # if there is a wake we calculate the velocity impact and the energy loss
                        U[(i, l)].append(j)
                        velocity[i, j, l] = calc_velocity(temp_y_i, temp_y_j, u_inf)
                        energy_coef[i, j, l] = 0.33 * (pow(u_inf, 3) - pow(velocity[i, j, l], 3))
                        ss_coef[i, j, l] = pow((1 - velocity[i, j, l] / wind_speeds[l]), 2)

    aggregated_coef = np.zeros((n, n))
    # prob_l = np.array([float(1)]*n_wind)
    # prob_l = np.true_divide(prob_l,n_wind)

    for i in range(n):
        for j in range(n):
            for l in range(0, len(wind_speeds)):
                if j in U[(i, l)]:
                    aggregated_coef[i, j] += prob_l[l] * energy_coef[i, j, l]

    return aggregated_coef, ss_coef, Epsilon, U, velocity

def qubo_fy(wind_speeds, prob_l, n, aggregated_coef, m):
    # scaling if needed due to embedding considerations
    scaling = 50  # 0
    arr_x = []
    exp_x = {}
    for i in range(n):
        arr_x.append(Binary('x' + str(i)))
        cur_coef = 0.33 * np.dot(prob_l, pow(wind_speeds, 3))

        # scaling:
        # cur_coef = scaling*(0.33*(pow(u_inf,3)))/max_energy

        # objective value for setting a turbine in position i
        exp_x[i] = cur_coef * arr_x[i]
        # print(cur_coef)

    for i in range(n):

        if i % 50 == 0:
            print(str(i) + ' out of ' + str(n))

        for j in range(n):
            # only basic velocity componenet
            if i != j:
                # no scaling:
                cur_coef = aggregated_coef[i, j]
                # scaling:
                # cur_coef = scaling*energy_coef[i,j]/max_energy
                # the influence of (i,j) coefficient in the objective
                exp_x[i] -= cur_coef * arr_x[i] * arr_x[j]
    exp = 0
    for i in range(n):
        exp += exp_x[i]
    # setting a penalty term
    lam_ = 10500
    # computing sum of decision variables
    sum_x = sum([arr_x[i] for i in range(n)])

    # constraint on the number of turbines
    exp2 = (sum_x - m) ** 2

    exp3 = 0
    for i in range(n):
        for j in range(n):
            # must not locate turbines too closely
            if (i, j) in Epsilon:
                exp3 += arr_x[i] * arr_x[j]

    # total objective function (minimizing)
    exp_total = -exp + lam_ * (exp2) + lam_ * (exp3)

    # qubo encoding into xQx-c
    qub_exp = exp_total.compile().to_qubo()
    return qub_exp
# In[11]:

def get_qubo(matrix, constr_constant):
    bias = np.zeros(N, dtype=int)
    constant = np.zeros(1, dtype=int)
    weight = np.zeros((N,N), dtype=int)
    state = np.zeros(N, dtype=int)

    matrix_size = matrix.shape[0]
    for i in range(matrix_size):
        for j in range(i, matrix_size):
            if i==j:
                bias[i] = -matrix[i,i]
            else:
                weight[i,j] = -matrix[i,j]
                weight[j,i] = -matrix[i,j]

    #decision variables>n must be set to -2^25 to make sure that these variables are not changed
    for i in range(matrix_size, 1024):
        bias[i] = 2**25

    weight = -weight
    bias = -bias
    constant[0] = constr_constant
    return weight, bias, constant, state


def check_matrix_sym(aggregated_coef):
    for i in range(n):
        for j in range(n):
            if i!=j:
                if aggregated_coef[i,j]!=aggregated_coef[j,i]:
                    print('assymetry detected')

# Important func -- Jason
def qubo_fy_eldan(wind_speeds, prob_l, n, aggregated_coef, m):
    matrix = np.zeros((n,n))
    # scaling if needed due to embedding considerations
    scaling = 50  # 0
    arr_x = []
    X = Array.create("x", n, 'BINARY')

    for i in range(n):
        matrix[i,i] = 0.33 * np.dot(prob_l, pow(wind_speeds, 3))


    for i in range(n):
        for j in range(i, n):
            matrix[i,j] -= aggregated_coef[i,j]
            matrix[j,i] -= matrix[i,j]

    # lam_ = 10500 is a large enough penalty coefficient
    # use a solution checker to make sure it is large enough
    lam_ = 10500
    # computing sum of decision variables

    # proximity constraint
    for (i, j) in Epsilon:
        matrix[i, j] -= lam_


    expr = lam_ * (sum(X) - m) ** 2
    # constraint on the number of turbines
    constr_coeffs, constr_constant = expr.compile().to_qubo()
    for i in range(n):
        for j in range(i, n):
            val = max(constr_coeffs.get((X[i].label, X[j].label), 0), constr_coeffs.get((X[j].label, X[i].label), 0))
            matrix[i, j] -= val

    return matrix, constr_constant

# call the solution checker in 'da_solve_params'
def solution_checker(state_min, Epsilon):
    # Check proximity constraint
    feas_proximity = 1
    for (i, j) in Epsilon:
        if state_min[i] == 1 and state_min[j] == 1:
            feas_proximity = 0
    
    return feas_proximity

def simulated_anneal(qub_exp, U):
    start_t = time.time()

    sampler = neal.SimulatedAnnealingSampler()
    response = QBSolv().sample_qubo(qub_exp[0], solver=sampler, num_repeats=100)
    # print("energies=" + str(list(response.data_vectors['energy'])))

    len(list(response.samples()))
    ll = list(response.samples())[len(list(response.samples())) - 1]
    count = 0
    relevant_sol = []
    for k, v in ll.items():
        if v > 0:
            # print(k)
            count += 1
            relevant_sol.append(int(str(k).split('x')[1]))
    print(count)

    total_energy = 0
    Energy = {}
    for i in relevant_sol:
        for l in range(len(wind_speeds)):
            # only wake!!!
            Energy[(i, l)] = 0.33 * prob_l[l]
            temp = 0
            for j in relevant_sol:
                if j in U[(i, l)]:
                    temp += ss_coef[i,j,l]#pow(1 - velocity[i, j, l] / wind_speeds[l], 2)
            Energy[(i, l)] *= pow(wind_speeds[l] * (1 - pow(temp, 0.5)), 3)

    total_energy = sum([Energy[(i, l)] for i in relevant_sol for l in range(len(wind_speeds))])
    print(total_energy)
    print('total time = ' + str(time.time() - start_t))
    return total_energy, time.time() - start_t

# In[ ]:



def calc_SS_energy(n, wind_speeds, prob_l, X, ss_coef, U):
    total_energy = 0
    Energy = {}
    N = range(n)
    for i in N:
        for l in range(len(wind_speeds)):
            # only wake!!!
            Energy[(i, l)] = 0.33 * prob_l[l] * X[i].x
            temp = 0
            for j in N:
                if j in U[(i, l)]:
                    temp += X[j].x * ss_coef[i,j,l] #pow(1 - velocity[i, j, l] / wind_speeds[l], 2)
            Energy[(i, l)] *= pow(wind_speeds[l] * (1 - pow(temp, 0.5)), 3)
    total_energy = sum([Energy[(i, l)] for i in N for l in range(len(wind_speeds))])
    print(total_energy)
    return total_energy



In [ ]:
da_obj = fjda.fjda()

In [ ]:
def da_solve_params(da, weights, bias, constant, init_state, velocity, m, num_iterations, offset):
    
    tmp_st = 500000
    tmp_interval = 1
    tmp_decay = 1 - (0.001 / tmp_st)**(1.0 / (float(num_iterations) / float(tmp_interval)))
    num_run = 20
    if num_iterations >= 100000000: #problematic should be 100000000
        num_run = 10
    
    # set anneal parameters
    start_t = time.time()
    pa = {'offset_inc_rate': offset,
          'tmp_st': tmp_st,
          'tmp_decay': tmp_decay,
          'tmp_mode': 0,
          'tmp_interval': tmp_interval,
          'noise_model': 0
         }
    
    # do anealling
    args = {
        'eg_start': constant.astype("int").tolist()[0],
        'state_i':  ''.join([chr(ord('0')+i) for i in init_state.astype("int").tolist()]),
        'bias':     bias.astype("int").tolist(),
        'weight':   weights.astype("int").reshape((1,1024*1024))[0].tolist(),
        #
        'num_bit':  1024,
        'num_iteration': num_iterations,
        'num_run':  num_run,
        'ope_mode': 1,
    }
    
    da.setAnnealParameter(pa)
    res = da.doAnneal(args)
    Obj = {}
    obj_da = 0.0
    # The following line is wrong!
    #obj_da -= res['eg_min_o_n'][0]
    time_info = da._rpc_time()
    print("Runtime: ", time_info)
    #convert qbits into list
    energy_max = []
    obj_max = []
    feasibility = []
    for i in range(num_run):
        state_min = [(ord(c)-ord('0')) for c in res['state_min_o_n'][i]] # '0010...'-->[0,0,1,0, ...]
        
        # Real Energy: Sum of Squares
        relevant_sol = state_min[0:(n)]
        total_energy = 0
        Energy = {}
        for i,r in enumerate(relevant_sol):
            for l in range(len(wind_speeds)):
            #only wake!!!
                Energy[(i,l)] = 0.33 * prob_l[l] * r
                temp = 0
                for j,o in enumerate(relevant_sol):
                    if j in U[(i,l)]:
                            temp += o * pow(1-velocity[i,j,l]/wind_speeds[l],2)
                Energy[(i,l)] *= pow(wind_speeds[l] * (1-pow(temp,0.5)) ,3) # SOM instead of LS
        total_energy = sum([Energy[(i,l)] for i in range(n) for l in range(len(wind_speeds))])        
        #print('Energy: ' + str(total_energy))
        energy_max.append(total_energy)
        
        for i,r in enumerate(relevant_sol):
            for l in range(len(wind_speeds)):
                Obj[(i,l)] = 0.33 * prob_l[l] * r
                temp2 = 0
                for j,o in enumerate(relevant_sol):
                    if j in U[(i,l)]:
                            temp2 += o * (pow(wind_speeds[l],3) - pow(velocity[i,j,l],3))
                Obj[(i,l)] *= pow(wind_speeds[l] ,3) - temp2 # Linear Superposition
        obj_da = sum([Obj[(i,l)] for i in range(n) for l in range(len(wind_speeds))])        
        obj_max.append(obj_da)
    obj_max_v = max(obj_max)
    max_idx = obj_max.index(max(obj_max))
    energy_max_v = energy_max[max_idx]
    
    print("Max energy: ", energy_max_v)
    print("Max index:", max_idx)
    print("Max objective: ", obj_max_v)
    
    return res, time_info, energy_max_v, obj_max_v, feasibility

In [ ]:
from collections import defaultdict

In [ ]:
# Please set your evironment to DA1 at first.

m_list = [10] #10,20,30,40,45,50,60,65,70,80,85,90,95,99,100
wind_list = [36]
axis_list = [10]
energies = []
config = []
runtimes_SA = []
MILP_Flag = False
QUBO_Gurobi_Flag = False
SA_Flag=False #True
DA_Flag = True
#Gurobi_exp = False

energy_results = defaultdict(lambda: dict())
obj_results = defaultdict(lambda: dict())
timing_results = defaultdict(lambda: dict())
feas_results = defaultdict(lambda: dict())

for n_wind in wind_list:
        for axis_ in axis_list:
# setting wind farm parameters
# n locations: every location has an x and a y coordinate
# For 2KMx2KM wind farm, and 100M per cell resolution we set a 20x20 grid with step size of 100M
#m = 1#20  # number of turbines
#axis_ = 2#10  # grid size (axis_ x axis_)
            n = pow(axis_, 2)  # cells
            if axis_==10:
                step = 200  # cell size in meters
            elif axis_==20:
                step =100
            else:
                step = 100

            angle = 360 / n_wind

            print('Calculating locations...')
            location_x , location_y = calc_location(axis_, step)
            print('Setting wind regime...')

            wind_speeds, prob_l = set_wind_regime(n_wind)
            print('Computing coefficients...')

            aggregated_coef, ss_coef, Epsilon, U, velocity = \
                calc_coefficients(location_x, location_y, wind_speeds, prob_l, n)
            #check_matrix_sym(aggregated_coef)
            for m in m_list:
                print('Number of turbines: '+str(m)) 

                if MILP_Flag ==True:
                    print('Running Gurobi MILP...')

                    #X =

                    energies.append(gurobi_MILP(n, wind_speeds, prob_l, m, aggregated_coef))

                    config.append((m,n,n_wind))
                    print(energies)
                    print(config)
                elif QUBO_Gurobi_Flag ==True:
                        print('Running Gurobi MILP...')

                        # X =

                        energies.append(gurobi_QUBO(n, wind_speeds, prob_l, m, aggregated_coef))
                        config.append((m, n, n_wind))
                        print(energies)
                        print(config)
                elif SA_Flag == True:
                    print('Running SA...')
                    SA_result = simulated_anneal(qubo_fy(wind_speeds, prob_l, n, aggregated_coef, m), U)
                    energies.append(SA_result[0])
                    runtimes_SA.append(SA_result[1])
                    config.append((m, n, n_wind))
                    print(energies)
                    print(config)
                    print(runtimes_SA)
                elif DA_Flag==True: # iters = 50000000 is the max
                    for iters in [1000000,3000000,5000000,7000000,10000000,20000000,30000000,40000000,50000000,80000000,100000000,140000000,200000000,250000000,300000000]:
                        for off in [10]: 
                            print("----")
                            print(iters, off)
                            matrix, const_coef = qubo_fy_eldan(wind_speeds, prob_l, n, aggregated_coef, m)
                            w, b, c, s = get_qubo(matrix, const_coef)
                            print('Constant: ', c)
                            res, timing, energy, obj_da, feas = da_solve_params(da_obj, w, b, c, s, velocity, m, num_iterations=iters, offset=off)
                            energy_results[(n_wind, axis_, m)][(iters, off)] = energy
                            obj_results[(n_wind, axis_, m)][(iters, off)] = obj_da
                            timing_results[(n_wind, axis_, m)][(iters, off)] = timing['elapsed_time']
                            feas_results[(n_wind, axis_, m)][(iters, off)] = feas
                            print("----") 


In [ ]:
import pandas as pd
iterate = [1000000,3000000,5000000,7000000,10000000,20000000,30000000,40000000,50000000,80000000,100000000,140000000,200000000,250000000,300000000]
for iterr in range(15):
    iteration = iterate[iterr]
    
    tuples = []
    fields = [(iteration, 10)]
    for inst in energy_results:
        tuples.append((str(inst),) + tuple([energy_results[inst][fld] for fld in fields]))

    energyDF = pd.DataFrame(tuples, columns = ["instance"] + [str(fld) for fld in fields])
    filename = 'energy_' + str(iteration) + '.csv'
    energyDF.to_csv(filename, index=False)
    
    tuples = []
    fields = [(iteration, 10)]
    for inst in obj_results:
        tuples.append((str(inst),) + tuple([obj_results[inst][fld] for fld in fields]))

    objDF = pd.DataFrame(tuples, columns = ["instance"] + [str(fld) for fld in fields])
    filename = 'obj_' + str(iteration) + '.csv'
    objDF.to_csv(filename, index=False)
    
    tuples = []
    fields = [(iteration, 10)]
    for inst in timing_results:
        tuples.append((str(inst),) + tuple([timing_results[inst][fld] for fld in fields]))

    timingDF = pd.DataFrame(tuples, columns = ["instance"] + [str(fld) for fld in fields])
    filename = 'time_' + str(iteration) + '.csv'
    timingDF.to_csv(filename, index=False)

In [ ]:
print("Finished!")